<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import plugin bổ sung của Folium

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Giải thích code: `spacex_df.groupby(['Launch Site'], as_index=False).first()`

👉 Gom nhóm dữ liệu theo từng **bãi phóng** (`Launch Site`).  
- `groupby(['Launch Site'])`: nhóm dữ liệu theo tên bãi phóng.  
- `first()`: lấy bản ghi đầu tiên của mỗi nhóm (vì mỗi bãi phóng có nhiều lần phóng).  
- `as_index=False`: để cột `Launch Site` vẫn là một cột bình thường, không bị biến thành index.  

📌 **Kết quả**: Mỗi bãi phóng chỉ xuất hiện **một lần** với giá trị Lat/Long tương ứng.


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# tạo bản đồ với tâm tại NASA Johnson Space Center (Houston, Texas):
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
site_map


We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [7]:
# Tạo vòng tròn xanh với popup
circle = folium.Circle(
    nasa_coordinate,         # tọa độ (lat, long) để vẽ vòng tròn
    radius=1000,             # bán kính vòng tròn = 1000m
    color='#d35400',         # màu viền vòng tròn (cam nâu)
    fill=True                # tô màu bên trong vòng tròn
).add_child(
    folium.Popup('NASA Johnson Space Center')  # khi click vào hiện popup
)

# Tạo marker có label text, tại cùng vị trí
marker = folium.map.Marker(
    nasa_coordinate,   # tọa độ đặt marker
    icon=DivIcon(      # dùng DivIcon để hiển thị text thay vì icon mặc định
        icon_size=(20,20),   # kích thước khung icon
        icon_anchor=(0,0),   # điểm neo icon
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)

# Thêm cả vòng tròn và marker text vào bản đồ site_map
site_map.add_child(circle)
site_map.add_child(marker)


In [8]:
site_map.save("map.html")


and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Khởi tạo bản đồ
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Với mỗi bãi phóng, thêm Circle & Marker
for index, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    name = row['Launch Site']

    # Circle
    circle = folium.Circle(coord, radius=1000, color='#000000', fill=True)\
                   .add_child(folium.Popup(name))
    site_map.add_child(circle)

    # Marker (label text)
    marker = folium.map.Marker(
        coord,
        icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % name,)
    )
    site_map.add_child(marker)


In [10]:
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


### Findings from the Launch Sites Map

**Q1. Are all launch sites in proximity to the Equator line?**  
- **Answer:** No.  
- The launch sites (VAFB in California, CCAFS LC-40 and KSC LC-39A in Florida) are located between approximately 28°–34° North latitude.  
- The Equator is at 0°, so these sites are not close to it.  
- However, Florida’s position (28°N) is relatively favorable compared to higher-latitude locations (like Europe or Russia), because it allows rockets to take better advantage of Earth’s rotational speed.

---

**Q2. Are all launch sites in very close proximity to the coast?**  
- **Answer:** Yes.  
- All the SpaceX launch sites are located near the coastline.  
- This ensures safety (rocket debris falls into the ocean instead of populated areas) and provides open access to various orbital trajectories over the sea.

---

**Conclusion:**  
- Launch sites are **not near the Equator**, but Florida offers some latitude advantage.  
- They are **all near the coast**, which is a common principle for spaceports worldwide.


### Kết quả từ bản đồ các bãi phóng

**Câu 1. Tất cả bãi phóng có gần đường xích đạo không?**  
- **Trả lời:** Không.  
- Các bãi phóng (VAFB ở California, CCAFS LC-40 và KSC LC-39A ở Florida) nằm ở khoảng vĩ độ 28°–34° Bắc.  
- Đường xích đạo ở 0°, nên chúng không gần.  
- Tuy vậy, vị trí Florida (28°B) có lợi thế hơn so với các khu vực vĩ độ cao khác (như châu Âu hoặc Nga) vì giúp tên lửa tận dụng tốt hơn vận tốc quay của Trái Đất.  

---

**Câu 2. Tất cả bãi phóng có nằm gần bờ biển không?**  
- **Trả lời:** Có.  
- Tất cả bãi phóng của SpaceX đều nằm sát bờ biển.  
- Điều này đảm bảo an toàn (mảnh vỡ rơi xuống biển thay vì khu dân cư) và thuận lợi cho việc triển khai nhiều loại quỹ đạo bay trên biển.  

---

**Kết luận:**  
- Các bãi phóng **không gần xích đạo**, nhưng Florida có lợi thế vĩ độ.  
- Tất cả **đều nằm sát bờ biển**, đây là nguyên tắc chung cho hầu hết các bãi phóng trên thế giới.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:
# Tạo cột mới 'marker_color' dựa trên giá trị cột 'class'
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Xem trước 5 dòng dữ liệu
spacex_df.head()


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df data frame
for index, record in spacex_df.iterrows():
    # Tạo Marker tại tọa độ bãi phóng
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],   # vị trí theo lat/long
        icon=folium.Icon(color=record['marker_color'])  # màu dựa vào cột marker_color
    )
    
    # Thêm Marker vào cụm marker_cluster
    marker_cluster.add_child(marker)

# Hiển thị bản đồ
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


## Task 3: Tính khoảng cách từ bãi phóng đến các khu vực lân cận  

### 🎯 Mục tiêu
- Phân tích xem các bãi phóng có gần **đường sắt, đường cao tốc, bờ biển** không.  
- Dùng công cụ **MousePosition** để lấy tọa độ (Lat, Long) khi rê chuột trên bản đồ.  
- Tính khoảng cách giữa bãi phóng và điểm gần nhất (coastline, highway, railway).  
- Vẽ **Polyline** để trực quan hóa khoảng cách.  


In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


## 🔎 Giải thích code

### 1. `formatter = "function(num) {return L.Util.formatNum(num, 5);}";`
- Đây là đoạn JavaScript inline, dùng để format số.  
- `L.Util.formatNum(num, 5)` → làm tròn số tới **5 chữ số thập phân**.  
- Kết quả: khi rê chuột trên bản đồ, tọa độ hiển thị gọn gàng (lat/long không quá dài).  

---

### 2. `mouse_position = MousePosition(...)`
- Tạo một đối tượng **MousePosition** để hiển thị tọa độ (lat, long) khi rê chuột.  
- Các tham số:
  - `position="topright"` → hiển thị box tọa độ ở góc trên bên phải bản đồ.  
  - `separator=" Long: "` → ký tự ngăn cách giữa Lat và Long.  
  - `empty_string="NaN"` → nếu chưa có dữ liệu thì hiển thị "NaN".  
  - `lng_first=False` → hiện Lat trước, Long sau.  
  - `num_digits=20` → số tối đa chữ số hiển thị, sau đó sẽ được định dạng lại.  
  - `prefix="lat:"` → trước vĩ độ sẽ có chữ "lat:".  
  - `lat_formatter=formatter, lng_formatter=formatter` → dùng function định dạng để làm tròn số.  

---

### 3. `site_map.add_child(mouse_position)`
- Thêm đối tượng `mouse_position` vào bản đồ `site_map`.  
- Kết quả: khi rê chuột trên bản đồ, góc trên phải sẽ hiển thị tọa độ (Lat, Long) theo định dạng đã chọn.  

---

### 4. `site_map`
- Gọi bản đồ để hiển thị (render trong Jupyter Notebook).  

---

## ✅ Tóm gọn
Đoạn code này giúp bạn **xem tọa độ (Lat, Long) của bất kỳ điểm nào trên bản đồ khi rê chuột qua**.  

- Lat/Long hiển thị ở góc trên phải.  
- Được format gọn đẹp (5 chữ số thập phân).  
- Rất hữu ích khi bạn muốn chọn các điểm mốc (coastline, highway, railway) để tính khoảng cách tới bãi phóng.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [17]:
# Lấy thông tin bãi phóng CCAFS LC-40
launch_site = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS LC-40'].iloc[0]

# Gán giá trị lat/lon vào biến
launch_site_lat = launch_site['Lat']
launch_site_lon = launch_site['Long']

# Tọa độ bờ biển gần nhất (ví dụ)
coastline_lat, coastline_lon = 28.5627, -80.56804  

# Tính khoảng cách
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon,
                                        coastline_lat, coastline_lon)
print("Distance:", distance_coastline, "km")


Distance: 0.9112284862253961 km


In [18]:
# Tạo marker hiển thị kết quả
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 
             ("{:.2f} KM".format(distance_coastline))
    )
)

site_map.add_child(distance_marker)

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [19]:
# Vẽ đường nối từ bãi phóng đến bờ biển
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon],
               [coastline_lat, coastline_lon]],
    weight=1
)
site_map.add_child(lines)


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [20]:
# Lấy tọa độ launch site: CCAFS LC-40
row = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS LC-40'].iloc[0]
launch_site_lat, launch_site_lon = row['Lat'], row['Long']

# Các POI bạn cung cấp
pois = [
    {"name": "Highway", "lat": 28.56249, "lon": -80.57068},
    {"name": "Railway", "lat": 28.56251, "lon": -80.58712},
    {"name": "City (Cape Canaveral)", "lat": 28.3936, "lon": -80.60329},
]

# Vẽ marker + khoảng cách + polyline cho từng POI
for p in pois:
    dist_km = calculate_distance(launch_site_lat, launch_site_lon, p["lat"], p["lon"])

    # Marker ghi khoảng cách tại điểm POI
    folium.Marker(
        [p["lat"], p["lon"]],
        tooltip=p["name"],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size:12px; color:#d35400;"><b>{p["name"]}: {dist_km:.2f} KM</b></div>'
        ),
    ).add_to(site_map)

    # Đường nối từ launch site -> POI
    folium.PolyLine(
        locations=[[launch_site_lat, launch_site_lon], [p["lat"], p["lon"]]],
        weight=2,  # nét vừa
        opacity=0.9
    ).add_to(site_map)

site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


### Findings about Launch Sites and Distances

- **Are launch sites near railways?**  
  Yes. The nearest railway is less than 1 km away (~0.95 km), indicating close proximity.

- **Are launch sites near highways?**  
  Yes. A highway is located very close (~0.65 km), showing convenient access.

- **Are launch sites near the coast?**  
  Yes. The sites are located right next to the coastline (~0.9 km), which is a common choice for safety and trajectory reasons.

- **Do launch sites maintain a certain distance from residential areas/cities?**  
  Yes. The nearest city (Cape Canaveral) is around 13–15 km away, which means the launch sites are separated from dense population areas for safety.


### Kết quả về các bãi phóng và khoảng cách

- **Các bãi phóng có nằm gần đường sắt không?**  
  Có. Đường sắt gần nhất cách dưới 1 km (~0.95 km), tức là khá gần.

- **Các bãi phóng có nằm gần đường cao tốc không?**  
  Có. Đường cao tốc nằm rất gần (~0.65 km), thuận tiện cho việc tiếp cận.

- **Các bãi phóng có nằm gần bờ biển không?**  
  Có. Các bãi phóng nằm sát bờ biển (~0.9 km), đây là lựa chọn phổ biến vì an toàn và thuận lợi cho quỹ đạo phóng.

- **Các bãi phóng có giữ khoảng cách nhất định với khu dân cư/thành phố không?**  
  Có. Thành phố gần nhất (Cape Canaveral) cách khoảng 13–15 km, cho thấy bãi phóng được đặt xa khu dân cư đông đúc để đảm bảo an toàn.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
